In [65]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

$P_{Fusion}(E,T) \propto \dfrac{1}{(kT)^{3/2}}\ \exp\left(-\dfrac{E}{kT}\right)\ \exp\left(-\dfrac{b}{\sqrt{E}}\right)$  
\
$T_C \approx 1.6\times10^7\ \left(\dfrac{R_0}{R}\right)\left(\dfrac{M}{M_0}\right)$

In [66]:
sun_radius = 6.957*10**8 #metres
sun_mass = 1.989*10**30 #kg
boltzmann = 1.380649*10**-23 #J/K
b = 10**-6

In [67]:
def central_temperature(star_radius, star_mass):

    return 1.6*10**7 * (sun_radius/star_radius) * (star_mass/sun_mass)

In [68]:
def plot(figure, figures=1):
    figure.update_layout(
        autosize=True,
        width=(1920/2) * figures,
        height=1080/2,
    )
    figure.show()

In [69]:
energy = np.array(np.linspace(1*10**-16, 100*10**-16, 100))
kT = boltzmann*central_temperature(sun_radius, sun_mass)
exp_1 = np.exp(-energy/kT)
exp_2 = np.exp(-b/energy**(1/2))
fusion_probability = (1/(kT**(3/2)))*exp_1*exp_2

In [70]:
fig = go.Figure()

fig.add_trace(go.Scatter(
        x=energy,
        y=exp_1,
        mode='markers',
        name=f""
))

fig.update_layout(
        title=f"",
        xaxis_title="",
        yaxis_title="",
        legend=dict(x=0.65, y=0.98)
)

plot(fig)

In [71]:
fig = go.Figure()

fig.add_trace(go.Scatter(
        x=energy,
        y=exp_2,
        mode='markers',
        name=f""
))

fig.update_layout(
        title=f"",
        xaxis_title="",
        yaxis_title="",
        legend=dict(x=0.65, y=0.98)
)

plot(fig)

In [72]:
fig = go.Figure()

fig.add_trace(go.Scatter(
        x=energy,
        y=fusion_probability,
        mode='markers',
        name=f""
))

fig.update_layout(
        title=f"",
        xaxis_title="",
        yaxis_title="",
        legend=dict(x=0.65, y=0.98)
)

plot(fig)